In [1]:
import os
import pandas as pd

# Read and concat yearly files
price_list, demand_list = [], []
for year in range(2014, 2026):
    p = pd.read_csv(
        fr'C:\Users\owner\Documents\Projects\hoep_forecasting_app\data\raw\PUB_PriceHOEPPredispOR_{year}.csv',
        skiprows=3
    )
    d = pd.read_csv(
        fr'C:\Users\owner\Documents\Projects\hoep_forecasting_app\data\raw\PUB_Demand_{year}.csv',
        skiprows=3
    )
    price_list.append(p); demand_list.append(d)

price_df  = pd.concat(price_list,  ignore_index=True)
demand_df = pd.concat(demand_list, ignore_index=True)

# Clean names
price_df.columns  = price_df.columns.str.strip()
demand_df.columns = demand_df.columns.str.strip()

price_df[['Hour 1 Predispatch', 'Hour 2 Predispatch', 'Hour 3 Predispatch']] = \
    price_df[['Hour 1 Predispatch', 'Hour 2 Predispatch', 'Hour 3 Predispatch']].ffill() # We use forward fill for NaNs as they appear in <1 percent of the data




for df in (price_df, demand_df):
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

    # Combine Date and Hour into naive datetime
    naive_ts = df['Date'] + pd.to_timedelta(df['Hour'] - 1, unit='h')

    # Localize with correct DST handling
    df['timestamp'] = naive_ts.dt.tz_localize(
        'Canada/Eastern',
        ambiguous=False,               # Convert to UTC to handle day light saving shifts in dataFrame
        nonexistent='shift_forward'   
    ).dt.tz_convert('UTC')



# Drop duplicates from daylight savings
price_df  = price_df.dropna(subset=['timestamp']).drop_duplicates(subset=['timestamp'])
demand_df = demand_df.dropna(subset=['timestamp']).drop_duplicates(subset=['timestamp'])

# Merge price + demand on timestamp
combined_df = pd.merge(
    price_df, demand_df,
    on='timestamp', how='inner',
    suffixes=('_price','_demand')
)
combined_df = combined_df.sort_values('timestamp').reset_index(drop=True)

# 6. Cleanup
combined_df = combined_df.drop(  # Drop non-live compatible features and duplicates columns
    columns=[
        'Date_price','Hour_price',
        'Date_demand','Hour_demand',
        'OR 10 Min Sync','OR 10 Min non-sync','OR 30 Min', 'Market Demand'
    ],
    errors='ignore'
)
combined_df = combined_df.set_index('timestamp')


print(f"\nFinal shape: {combined_df.shape}")
print(combined_df.isna().sum())
print("Range:", combined_df.index.min(), "→", combined_df.index.max()) # Final output (number of NaNs for each column, range, and shape)


Final shape: (99300, 5)
HOEP                  0
Hour 1 Predispatch    0
Hour 2 Predispatch    0
Hour 3 Predispatch    0
Ontario Demand        0
dtype: int64
Range: 2014-01-01 05:00:00+00:00 → 2025-05-01 03:00:00+00:00


In [2]:
import os
import pandas as pd

# Load weather folder paths
root = r'C:\Users\owner\Documents\Projects\hoep_forecasting_app\data\raw\weather'
cities = ['toronto','kitchener','london','ottawa']
col_map = {
    "Temp (°C)":    "temp",
    "Rel Hum (%)":  "humidity",
    "Wind Spd (km/h)": "wind_speed",
}

city_dfs = []
for city in cities:
    path = os.path.join(root, city) # Becomes raw/weather/toronto , kitchener etc..
    dfs = []
    for fname in sorted(os.listdir(path)):  # Sort from 2014-2025
        df = pd.read_csv(os.path.join(path, fname))
        df.columns = df.columns.str.strip().str.replace('"','') # Clean columns

        naive_ts = pd.to_datetime(df['Date/Time (LST)'], errors='coerce')

        df = df[list(col_map)].rename(columns=col_map) # Rename wanted columns and drop unwanted ones


        df['timestamp'] = naive_ts.dt.tz_localize('Canada/Eastern', ambiguous=False, nonexistent='shift_forward').dt.tz_convert('UTC') # UTC timestamp 
        dfs.append(df)
    
   
    city_df = pd.concat(dfs, ignore_index=True)
    city_df  = city_df.drop_duplicates(subset=['timestamp']) # Drop daylight saving timestamps, we use shift forward to create duplicates and then drop them
    city_df = city_df.set_index('timestamp').sort_index()
    
    city_df = city_df.ffill() # We use forward fill as close to 1 percent data missing. Ffil is sufficent
    city_df = city_df.add_suffix(f"_{city}")
    city_dfs.append(city_df) # Contains each cities sorted cleaned and filled df

# inner join across all cities (keeps only timestamps present in every city)
from functools import reduce
weather_df = reduce(lambda L, R: L.join(R, how='inner'), city_dfs)
weather_df = weather_df.sort_index()

print("Shape:", weather_df.shape)
print("Range:", weather_df.index.min(), "→", weather_df.index.max())  # Shape and ranges matches previous cell df
 


Shape: (99300, 12)
Range: 2014-01-01 05:00:00+00:00 → 2025-05-01 03:00:00+00:00


In [7]:
# Both have 'timestamp' index in UTC now, accounts for daylight savings
final_df = combined_df.join(weather_df, how='inner')

print("\nMerged shape:", final_df.shape)
print("Merged range:", final_df.index.min(), final_df.index.max())


print(final_df.isna().sum())
# Final Nan check , range and shape check


Merged shape: (99300, 17)
Merged range: 2014-01-01 05:00:00+00:00 2025-05-01 03:00:00+00:00
HOEP                    0
Hour 1 Predispatch      0
Hour 2 Predispatch      0
Hour 3 Predispatch      0
Ontario Demand          0
temp_toronto            0
humidity_toronto        0
wind_speed_toronto      0
temp_kitchener          0
humidity_kitchener      0
wind_speed_kitchener    0
temp_london             0
humidity_london         0
wind_speed_london       0
temp_ottawa             0
humidity_ottawa         0
wind_speed_ottawa       0
dtype: int64


In [67]:
# We create the Dataframe for the ELM. We use a minimum shift of 2 on all features except IESO predispatch and time features. These are forecasts known at the time of prediction
import numpy as np


# Config
hoep_lags = [1, 2, 3, 6, 12]

# Reset index to create time features using timestamp

PDP_df = final_df.copy() # Copy unshifted DF for PDP metrics
df = final_df.copy()
df = df.reset_index()

# Convert from string to numeric
df['HOEP'] = pd.to_numeric(df['HOEP'].astype(str).str.replace(',', ''), errors='coerce')
df['Hour 1 Predispatch'] = pd.to_numeric(df['Hour 1 Predispatch'].astype(str).str.replace(',', ''), errors='coerce')
df['Hour 2 Predispatch'] = pd.to_numeric(df['Hour 2 Predispatch'].astype(str).str.replace(',', ''), errors='coerce')
df['Hour 3 Predispatch'] = pd.to_numeric(df['Hour 3 Predispatch'].astype(str).str.replace(',', ''), errors='coerce')



df['Hour 1 Predispatch'] = df['Hour 1 Predispatch'].shift(-1)
df['Hour 2 Predispatch'] = df['Hour 2 Predispatch'].shift(-2)
df['Hour 3 Predispatch'] = df['Hour 3 Predispatch'].shift(-3)

df['price_forecast_error_h1'] = df['HOEP'] - df['Hour 1 Predispatch'].shift(1)
df['price_forecast_error_h2'] = df['HOEP'] - df['Hour 2 Predispatch'].shift(2)  
df['price_forecast_error_h3'] = df['HOEP'] - df['Hour 3 Predispatch'].shift(3)

# Change to datetime object for feature engineering
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Time features
df['month'] = df['timestamp'].dt.month 
df['hour'] = df['timestamp'].dt.hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)





lag_features = {}
for k in hoep_lags:
    lag_features[f'HOEP_lag_{k}']= df['HOEP'].shift(k)

df = pd.concat([df, pd.DataFrame(lag_features, index=df.index)], axis=1)

# Final cleanup
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
df = df.set_index('timestamp').sort_index()
df = df.dropna()

# Drop leakage columns
cols_to_drop = [      
   'hour', 
   'month',
   'temp_toronto',
   'wind_speed_toronto', 
   'humidity_toronto',
   'temp_london',
   'wind_speed_london',
   'humidity_london', 
   'temp_kitchener',
   'wind_speed_kitchener',
   'humidity_kitchener',
   'temp_ottawa',
   'wind_speed_ottawa',
   'humidity_ottawa'
]
df.drop(columns=cols_to_drop, inplace=True)
print(f"Final shape: {df.shape}")
features = df.columns.tolist()
print(features)


Final shape: (99285, 17)
['HOEP', 'Hour 1 Predispatch', 'Hour 2 Predispatch', 'Hour 3 Predispatch', 'Ontario Demand', 'price_forecast_error_h1', 'price_forecast_error_h2', 'price_forecast_error_h3', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'HOEP_lag_1', 'HOEP_lag_2', 'HOEP_lag_3', 'HOEP_lag_6', 'HOEP_lag_12']


In [68]:
# Create multi-horizon targets (t+1, t+2, t+3)
df['target_t1'] = df['HOEP'].shift(-1)  # 1 hour ahead
df['target_t2'] = df['HOEP'].shift(-2)  # 2 hours ahead  
df['target_t3'] = df['HOEP'].shift(-3)  # 3 hours ahead

targets = ['target_t1', 'target_t2', 'target_t3']

df = df.dropna(subset=targets)

# 3-way split for proper validation
train_start = pd.Timestamp("2014-01-01 00:00:00", tz="UTC")
train_cutoff = pd.Timestamp("2023-01-01 00:00:00", tz="UTC")
val_cutoff = pd.Timestamp("2024-01-01 00:00:00", tz="UTC")
final_cutoff = pd.Timestamp("2025-01-01 00:00:00", tz='UTC')

df_train = df[(df.index < train_cutoff)] # Train on 2014-2022
df_val = df[(df.index >= train_cutoff) & (df.index < val_cutoff)]  # Validate on 2023
df_test = df[(df.index >= val_cutoff ) & (df.index < final_cutoff)] # Test on 2024


X_train = df_train[features].apply(pd.to_numeric, errors="coerce")
y_train = df_train[targets].apply(pd.to_numeric, errors="coerce")

X_val = df_val[features].apply(pd.to_numeric, errors="coerce")
y_val = df_val[targets].apply(pd.to_numeric, errors="coerce")

X_test = df_test[features].apply(pd.to_numeric, errors="coerce")
y_test = df_test[targets].apply(pd.to_numeric, errors="coerce")

# Save column names before scaling
feature_names_ELM = X_train.columns.tolist()



In [69]:
import time
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np
import pandas as pd

np.random.seed(42)

def calculate_multi_horizon_metrics(y_true, y_pred, horizon_names=['t+1', 't+2', 't+3']):
    metrics = {}
    
    for i, horizon in enumerate(horizon_names):
        y_true_horizon = y_true.iloc[:, i] if hasattr(y_true, 'iloc') else y_true[:, i]
        y_pred_horizon = y_pred[:, i] if y_pred.ndim > 1 else y_pred
        
        mae = mean_absolute_error(y_true_horizon, y_pred_horizon)
        rmse = np.sqrt(mean_squared_error(y_true_horizon, y_pred_horizon))
        mape = np.mean(np.abs((y_true_horizon - y_pred_horizon) / y_true_horizon)) * 100
        r2 = r2_score(y_true_horizon, y_pred_horizon)
        
        metrics[f'{horizon}_MAE'] = mae
        metrics[f'{horizon}_RMSE'] = rmse
        metrics[f'{horizon}_MAPE'] = mape
        metrics[f'{horizon}_R2'] = r2
    
    overall_mae = np.mean([metrics[f'{h}_MAE'] for h in horizon_names])
    overall_rmse = np.mean([metrics[f'{h}_RMSE'] for h in horizon_names])
    overall_r2 = np.mean([metrics[f'{h}_R2'] for h in horizon_names])
    
    metrics['Overall_MAE'] = overall_mae
    metrics['Overall_RMSE'] = overall_rmse
    metrics['Overall_R2'] = overall_r2
    
    return metrics

def print_metrics(metrics, title="Metrics"):
    print(f"\n{title}:")
    
    horizons = ['t+1', 't+2', 't+3']
    for horizon in horizons:
        print(f"{horizon} - RMSE: {metrics[f'{horizon}_RMSE']:.2f}, "
              f"MAE: {metrics[f'{horizon}_MAE']:.2f}, "
              f"R²: {metrics[f'{horizon}_R2']:.3f}")
    
    print(f"Overall - RMSE: {metrics['Overall_RMSE']:.2f}, "
          f"MAE: {metrics['Overall_MAE']:.2f}, "
          f"R²: {metrics['Overall_R2']:.3f}")

best_rmse = float('inf')
best_n_hidden = None
best_k = None
best_selector = None
best_train_time = None
best_scaler = None
best_selected_feature_names = None

for k in [len(X_train.columns)]:
    print(f"\n Testing with k={k} features ")
    
    if k == len(X_train.columns):
        X_train_sel_raw = X_train
        X_val_sel_raw = X_val
        X_test_sel_raw = X_test
        selector = None
        selected_features = X_train.columns
    else:
        selector = SelectKBest(score_func=f_regression, k=k)
        y_train_avg = y_train.mean(axis=1)
        selector.fit(X_train, y_train_avg)
        
        X_train_sel_raw = selector.transform(X_train)
        X_val_sel_raw = selector.transform(X_val)
        X_test_sel_raw = selector.transform(X_test)
        selected_features = X_train.columns[selector.get_support()]
    
    scaler = StandardScaler()
    X_train_sel = scaler.fit_transform(X_train_sel_raw)
    X_val_sel = scaler.transform(X_val_sel_raw)
    X_test_sel = scaler.transform(X_test_sel_raw)
    
    print("Hyperparameter Tuning Results:")
    
    for n_hidden in [100, 250, 500, 750, 1000]:
        start_time = time.time()
        
        W_in = np.random.randn(X_train_sel.shape[1], n_hidden)
        H_train = np.maximum(0, X_train_sel @ W_in)
        H_val = np.maximum(0, X_val_sel @ W_in)
        
        W_out = np.linalg.pinv(H_train) @ y_train.values
        
        train_time = time.time() - start_time
        
        y_val_pred = H_val @ W_out
        
        val_metrics = calculate_multi_horizon_metrics(y_val, y_val_pred)
        val_rmse = val_metrics['Overall_RMSE']
        
        print(f"n_hidden={n_hidden}: Overall Val RMSE = {val_rmse:.2f}, Train Time = {train_time:.3f}s")
        
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            best_n_hidden = n_hidden
            best_k = k
            best_selector = selector
            best_scaler = scaler
            best_train_time = train_time
            best_selected_feature_names = selected_features

print("\nFinal Best ELM Model:")
print(f"Best k (num features): {best_k}")
print(f"Best n_hidden        : {best_n_hidden}")
print(f"Best validation RMSE : {best_rmse:.2f}")
print(f"Train Time           : {best_train_time:.3f}s")


print("\nFINAL MODEL EVALUATION")

np.random.seed(42)

if best_selector is not None:
    X_train_sel_raw = best_selector.transform(X_train)
    X_test_sel_raw = best_selector.transform(X_test)
else:
    X_train_sel_raw = X_train
    X_test_sel_raw = X_test

X_train_sel = best_scaler.fit_transform(X_train_sel_raw)
X_test_sel = best_scaler.transform(X_test_sel_raw)

W_in = np.random.randn(X_train_sel.shape[1], best_n_hidden)
H_train = np.maximum(0, X_train_sel @ W_in)
H_test = np.maximum(0, X_test_sel @ W_in)
W_out = np.linalg.pinv(H_train) @ y_train.values

y_test_pred = H_test @ W_out

test_metrics = calculate_multi_horizon_metrics(y_test, y_test_pred)
print_metrics(test_metrics, "Final Test Results")

model_data = {
    "W_in": W_in,
    "W_out": W_out,
    "n_hidden": best_n_hidden,
    "scaler": best_scaler,
    "selector": best_selector,
    "feature_names": best_selected_feature_names,
    "horizons": ['t+1', 't+2', 't+3']
}

joblib.dump(model_data, "elm_multi_horizon_model.pkl")


prediction_results = {}
for i, horizon in enumerate(['t+1', 't+2', 't+3']):
    y_test_horizon = pd.Series(y_test.iloc[:, i].values, index=df_test.index, name=f'Actual_{horizon}')
    y_pred_horizon = pd.Series(y_test_pred[:, i], index=df_test.index, name=f'Predicted_{horizon}')
    
    prediction_results[horizon] = {
        'actual': y_test_horizon,
        'predicted': y_pred_horizon
    }




 Testing with k=17 features 
Hyperparameter Tuning Results:
n_hidden=100: Overall Val RMSE = 16.27, Train Time = 1.424s
n_hidden=250: Overall Val RMSE = 15.91, Train Time = 3.640s
n_hidden=500: Overall Val RMSE = 15.78, Train Time = 8.476s
n_hidden=750: Overall Val RMSE = 15.98, Train Time = 14.266s
n_hidden=1000: Overall Val RMSE = 16.03, Train Time = 23.691s

Final Best ELM Model:
Best k (num features): 17
Best n_hidden        : 500
Best validation RMSE : 15.78
Train Time           : 8.476s

FINAL MODEL EVALUATION

Final Test Results:
t+1 - RMSE: 22.49, MAE: 7.78, R²: 0.308
t+2 - RMSE: 23.89, MAE: 8.82, R²: 0.220
t+3 - RMSE: 24.42, MAE: 9.28, R²: 0.185
Overall - RMSE: 23.60, MAE: 8.63, R²: 0.238


In [70]:
import joblib
import json
import numpy as np

# Load the saved ELM model
model = joblib.load("elm_multi_horizon_model.pkl")

# Extract and convert weights + scaler
W_in = model["W_in"].tolist()         # shape: [n_features, n_hidden]
W_out = model["W_out"].tolist()       # shape: [n_hidden] or [n_hidden, 1]
scaler_mean = model["scaler"].mean_.tolist()
scaler_scale = model["scaler"].scale_.tolist()
n_hidden = model["n_hidden"]

# Prepare dictionary
model_json = {
    "W_in": W_in,
    "W_out": W_out,
    "scaler_mean": scaler_mean,
    "scaler_scale": scaler_scale,
    "n_hidden": n_hidden
}

# Save to JSON
with open("elm_model_extracted.json", "w") as f:
    json.dump(model_json, f)

print(" Model successfully exported to elm_model_extracted.json")


 Model successfully exported to elm_model_extracted.json


In [71]:
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

n_steps = 12
batch_size = 64
epochs = 50
lstm_units = 64

print("Preparing LSTM data using same train/val/test splits as ELM...")
print(f"Train period: {df_train.index.min()} to {df_train.index.max()}")
print(f"Val period: {df_val.index.min()} to {df_val.index.max()}")
print(f"Test period: {df_test.index.min()} to {df_test.index.max()}")

# Create multi-horizon targets
def create_multi_horizon_targets(df_subset):
    df_copy = df_subset.copy()
    df_copy['target_t1'] = df_copy['HOEP'].shift(-1)
    df_copy['target_t2'] = df_copy['HOEP'].shift(-2)
    df_copy['target_t3'] = df_copy['HOEP'].shift(-3)
    df_copy = df_copy.dropna(subset=['target_t1', 'target_t2', 'target_t3'])
    return df_copy

lstm_train = create_multi_horizon_targets(df_train)
lstm_val = create_multi_horizon_targets(df_val)
lstm_test = create_multi_horizon_targets(df_test)

print(f"Train shape: {lstm_train.shape}")
print(f"Val shape: {lstm_val.shape}")
print(f"Test shape: {lstm_test.shape}")

# Use same features as ELM but exclude HOEP lags for LSTM
lag_columns = [col for col in features if 'HOEP_lag_' in col]
lstm_features = [col for col in features if col not in ['target_t1', 'target_t2', 'target_t3'] + lag_columns]
targets = ['target_t1', 'target_t2', 'target_t3']


X_train_lstm_raw = lstm_train[lstm_features].values
X_val_lstm_raw = lstm_val[lstm_features].values
X_test_lstm_raw = lstm_test[lstm_features].values

y_train_lstm = lstm_train[targets].values
y_val_lstm = lstm_val[targets].values
y_test_lstm = lstm_test[targets].values

feature_scaler = StandardScaler()
X_train_lstm = feature_scaler.fit_transform(X_train_lstm_raw)
X_val_lstm = feature_scaler.transform(X_val_lstm_raw)
X_test_lstm = feature_scaler.transform(X_test_lstm_raw)

target_scaler = StandardScaler()
y_train_lstm_scaled = target_scaler.fit_transform(y_train_lstm)
y_val_lstm_scaled = target_scaler.transform(y_val_lstm)
y_test_lstm_scaled = target_scaler.transform(y_test_lstm)



def create_sequences(X, y, n_steps):
    X_seq, y_seq = [], []
    for i in range(n_steps, len(X)):
        X_seq.append(X[i - n_steps:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)



X_train_seq, y_train_seq = create_sequences(X_train_lstm, y_train_lstm_scaled, n_steps)
X_val_seq, y_val_seq = create_sequences(X_val_lstm, y_val_lstm_scaled, n_steps)
X_test_seq, y_test_seq = create_sequences(X_test_lstm, y_test_lstm_scaled, n_steps)

print(f"Sequence shapes:")
print(f"X_train_seq: {X_train_seq.shape}")
print(f"y_train_seq: {y_train_seq.shape}")
print(f"X_val_seq: {X_val_seq.shape}")
print(f"X_test_seq: {X_test_seq.shape}")

tf.random.set_seed(42)

model = Sequential([
    LSTM(lstm_units, input_shape=(n_steps, X_train_seq.shape[2]), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(3)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss='mse',
    metrics=['mae']
)

print("\nModel Architecture:")
model.summary()

early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001,
    verbose=1
)

callbacks = [early_stop, reduce_lr]

print(f"\nTraining LSTM model...")
start_time = time.time()

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    shuffle=False,
    verbose=1
)

train_time = time.time() - start_time


y_val_pred_scaled = model.predict(X_val_seq)
y_test_pred_scaled = model.predict(X_test_seq)

y_val_pred = target_scaler.inverse_transform(y_val_pred_scaled)
y_test_pred = target_scaler.inverse_transform(y_test_pred_scaled)

y_val_true = target_scaler.inverse_transform(y_val_seq)
y_test_true = target_scaler.inverse_transform(y_test_seq)

def calculate_multi_horizon_metrics(y_true, y_pred, horizon_names=['t+1', 't+2', 't+3']):
    metrics = {}
    
    for i, horizon in enumerate(horizon_names):
        y_true_horizon = y_true[:, i]
        y_pred_horizon = y_pred[:, i]
        
        mae = mean_absolute_error(y_true_horizon, y_pred_horizon)
        rmse = np.sqrt(mean_squared_error(y_true_horizon, y_pred_horizon))
        r2 = r2_score(y_true_horizon, y_pred_horizon)
        
        metrics[f'{horizon}_MAE'] = mae
        metrics[f'{horizon}_RMSE'] = rmse
        metrics[f'{horizon}_R2'] = r2
    
    overall_mae = np.mean([metrics[f'{h}_MAE'] for h in horizon_names])
    overall_rmse = np.mean([metrics[f'{h}_RMSE'] for h in horizon_names])
    overall_r2 = np.mean([metrics[f'{h}_R2'] for h in horizon_names])
    
    metrics['Overall_MAE'] = overall_mae
    metrics['Overall_RMSE'] = overall_rmse
    metrics['Overall_R2'] = overall_r2
    
    return metrics

def print_metrics(metrics, title="Metrics"):
    print(f"\n{title}:")
    print("="*50)
    horizons = ['t+1', 't+2', 't+3']
    for horizon in horizons:
        print(f"{horizon} - RMSE: {metrics[f'{horizon}_RMSE']:.2f}, "
              f"MAE: {metrics[f'{horizon}_MAE']:.2f}, "
              f"R²: {metrics[f'{horizon}_R2']:.3f}")
    
    print("-"*50)
    print(f"Overall - RMSE: {metrics['Overall_RMSE']:.2f}, "
          f"MAE: {metrics['Overall_MAE']:.2f}, "
          f"R²: {metrics['Overall_R2']:.3f}")

val_metrics = calculate_multi_horizon_metrics(y_val_true, y_val_pred)
print_metrics(val_metrics, "LSTM Validation Results")

test_metrics = calculate_multi_horizon_metrics(y_test_true, y_test_pred)
print_metrics(test_metrics, "LSTM Test Results")

print(f"\nTraining Summary:")
print(f"Train Time: {train_time:.2f} seconds")
print(f"Total Epochs: {len(history.history['loss'])}")
print(f"Best Val Loss: {min(history.history['val_loss']):.4f}")

print(f"\nSaving model...")
model.save("lstm_multi_horizon_model.keras")


lstm_results = {
    'val_metrics': val_metrics,
    'test_metrics': test_metrics,
    'train_time': train_time,
    'model_params': {
        'n_steps': n_steps,
        'lstm_units': lstm_units,
        'epochs_trained': len(history.history['loss']),
        'batch_size': batch_size
    }
}



Preparing LSTM data using same train/val/test splits as ELM...
Train period: 2014-01-01 17:00:00+00:00 to 2022-12-31 23:00:00+00:00
Val period: 2023-01-01 00:00:00+00:00 to 2023-12-31 23:00:00+00:00
Test period: 2024-01-01 00:00:00+00:00 to 2024-12-31 23:00:00+00:00
Train shape: (78859, 20)
Val shape: (8756, 20)
Test shape: (8780, 20)
Sequence shapes:
X_train_seq: (78847, 12, 12)
y_train_seq: (78847, 3)
X_val_seq: (8744, 12, 12)
X_test_seq: (8768, 12, 12)


c:\Users\owner\Documents\Projects\research\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Model Architecture:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,891 (85.51 KB)

 Trainable params: 21,891 (85.51 KB)

 Non-trainable params: 0 (0.00 B)


Training LSTM model...
Epoch 1/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 1.0614 - mae: 0.4158 - val_loss: 0.2417 - val_mae: 0.2111 - learning_rate: 0.0010
Epoch 2/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - loss: 0.8186 - mae: 0.3346 - val_loss: 0.2350 - val_mae: 0.2003 - learning_rate: 0.0010
Epoch 3/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - loss: 0.7917 - mae: 0.3234 - val_loss: 0.2361 - val_mae: 0.2057 - learning_rate: 0.0010
Epoch 4/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - loss: 0.7785 - mae: 0.3171 - val_loss: 0.2344 - val_mae: 0.2036 - learning_rate: 0.0010
Epoch 5/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 0.7716 - mae: 0.3153 - val_loss: 0.2328 - val_mae: 0.1981 - learning_rate: 0.0010
Epoch 6/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - loss: 0.7522 - mae: 0.3099 - val_loss: 0.2360 - val_mae: 0.2050 - learning_rate: 0.0010
Epoch 7/50
1232/1232 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - loss: 0.7473 - mae: 0.3105 - val_loss: 0.2

In [ ]:
# Print baseline Metrics from IESO, 2 and 3 hour forecasted RMSE, MAE on 2024 data


from sklearn.metrics import mean_squared_error, mean_absolute_error

target = 'HOEP'


# 3-way split for proper validation
train_start = pd.Timestamp("2014-01-01 00:00:00", tz="UTC")
train_cutoff = pd.Timestamp("2023-01-01 00:00:00", tz="UTC")
val_cutoff = pd.Timestamp("2024-01-01 00:00:00", tz="UTC")
final_cutoff = pd.Timestamp("2025-01-01 00:00:00", tz='UTC')

df_train = PDP_df[(PDP_df.index < train_cutoff)] # Train on 2014-2022
df_val = PDP_df[(PDP_df.index >= train_cutoff) & (PDP_df.index < val_cutoff)]  # Validate on 2023
df_test = PDP_df[(PDP_df.index >= val_cutoff ) & (PDP_df.index < final_cutoff)] # Test on 2024


y_test = df_test[target].apply(pd.to_numeric, errors="coerce")

# Hour 1
rmse_1 = np.sqrt(mean_squared_error(y_test, df_test["Hour 1 Predispatch"]))
mae_1  = mean_absolute_error(y_test, df_test["Hour 1 Predispatch"])
# Hour 2
rmse_2 = np.sqrt(mean_squared_error(y_test, df_test["Hour 2 Predispatch"]))
mae_2  = mean_absolute_error(y_test, df_test["Hour 2 Predispatch"])

# Hour 3
rmse_3 = np.sqrt(mean_squared_error(y_test, df_test["Hour 3 Predispatch"]))
mae_3  = mean_absolute_error(y_test, df_test["Hour 3 Predispatch"])



print(f"Hour 1 Predispatch RMSE: {rmse_1:.2f} CAD/MWh")
print(f"Hour 1 Predispatch MAE : {mae_1:.2f} CAD/MWh")
print(f"Hour 2 Predispatch RMSE: {rmse_2:.2f} CAD/MWh")
print(f"Hour 2 Predispatch MAE : {mae_2:.2f} CAD/MWh")
print(f"Hour 3 Predispatch RMSE: {rmse_3:.2f} CAD/MWh")
print(f"Hour 3 Predispatch MAE : {mae_3:.2f} CAD/MWh")





Hour 1 Predispatch RMSE: 32.14 CAD/MWh
Hour 1 Predispatch MAE : 11.68 CAD/MWh
Hour 2 Predispatch RMSE: 29.67 CAD/MWh
Hour 2 Predispatch MAE : 11.44 CAD/MWh
Hour 3 Predispatch RMSE: 28.38 CAD/MWh
Hour 3 Predispatch MAE : 10.54 CAD/MWh
